<a href="https://colab.research.google.com/github/Arzuki-zheng/border-collie-classifier/blob/main/notebooks/border_collie_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>